## Example using LSTM network for sentiment classification on IMDB data

Source: https://keras.io/examples/imdb_lstm/

It looks like the architecture here will be similar to our project, but the dimenisons of the input and output and data will be different.

About the data:

"Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer '3' encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: 'only consider the top 10,000 most common words, but eliminate the top 20 most common words'.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word."

In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [4]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

How each sentence is represented:

In [6]:
print(x_train[0])

[   15   256     4     2     7  3766     5   723    36    71    43   530
   476    26   400   317    46     7     4 12118  1029    13   104    88
     4   381    15   297    98    32  2071    56    26   141     6   194
  7486    18     4   226    22    21   134   476    26   480     5   144
    30  5535    18    51    36    28   224    92    25   104     4   226
    65    16    38  1334    88    12    16   283     5    16  4472   113
   103    32    15    16  5345    19   178    32]


In [5]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


This example uses the embedding layer to learn a word embedding within the model. This takes the vocabulary size and desired embedding dimension as arguments. Next an LSTM layer with dropout is added. The final dense layer outputs the classification, with dimension 1.

In [7]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


This takes about 30 minutes to train. The example originally trained for 15 epochs, but I scaled that back to 5.

In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...


C:\Users\Chloe\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 371s 15ms/step - loss: 0.4543 - accuracy: 0.7877 - val_loss: 0.3607 - val_accuracy: 0.8400
Epoch 2/5
25000/25000 [==============================] - 361s 14ms/step - loss: 0.2982 - accuracy: 0.8820 - val_loss: 0.3726 - val_accuracy: 0.8380
Epoch 3/5
25000/25000 [==============================] - 353s 14ms/step - loss: 0.2179 - accuracy: 0.9159 - val_loss: 0.4520 - val_accuracy: 0.8122
Epoch 4/5
25000/25000 [==============================] - 429s 17ms/step - loss: 0.1577 - accuracy: 0.9412 - val_loss: 0.4608 - val_accuracy: 0.8308
Epoch 5/5
25000/25000 [==============================] - 83s 3ms/step
Test score: 0.5362590254974365
Test accuracy: 0.8199999928474426
